In [1]:
import pandas as pd
import geopandas as gpd
from fuzzywuzzy import process

In [2]:
df = pd.read_csv('data/data_macroeconomy.csv')

In [4]:
df.head()

,Kab/Kota,Tahun,Regional GDP (IDR Bil),Regional GDP per Capita
0,SIMEULUE,2022,"1,410.00","14,960,053.00"
1,ACEH SINGKIL,2022,"1,539.00","11,908,999.00"
2,ACEH SELATAN,2022,"3,743.00","15,838,559.00"
3,ACEH TENGGARA,2022,"3,054.00","13,436,165.00"
4,ACEH TIMUR,2022,"7,162.00","16,710,999.00"


In [11]:
df['Regional GDP (IDR Bil)'] = df['Regional GDP (IDR Bil)'].str.replace(',', '', regex=False).astype(float).astype(int)
df['Regional GDP per Capita'] = df['Regional GDP per Capita'].str.replace(',', '', regex=False).astype(float).astype(int)

In [12]:
df.dtypes

Kab/Kota                   object
Tahun                       int64
Regional GDP (IDR Bil)      int32
Regional GDP per Capita     int32
dtype: object

In [13]:
df['Kab/Kota'] = df['Kab/Kota'].str.upper()

In [14]:
# Baca geojson
gdf = gpd.read_file('data/indonesia.geojson')

In [15]:
gdf['state'] = gdf['state'].str.upper()

In [16]:
nama_geo = list(gdf['state'].unique())

In [17]:
mapping = {kab: process.extractOne(kab, nama_geo)[0] for kab in df['Kab/Kota'].unique()}

In [18]:
df['geo_name'] = df['Kab/Kota'].map(mapping)

In [19]:
# Filter untuk tahun 2022
df2022 = df[df['Tahun']==2022]
merged = gdf.merge(df2022, left_on='state', right_on='geo_name')
merged.to_file('data/merged_2022.geojson', driver='GeoJSON')

In [20]:
gdf_1 = gpd.read_file('data/merged_2022.geojson')

In [21]:
gdf_1.head()

,cartodb_id,country,id_1,slug,state,Kab/Kota,Tahun,Regional GDP (IDR Bil),Regional GDP per Capita,geo_name,geometry
0,16,Indonesia,1,indonesia-aceh,ACEH,ACEH SINGKIL,2022,1539,11908999,ACEH,"MULTIPOLYGON (((97.97681 4.62750, 98.00173 4.6..."
1,16,Indonesia,1,indonesia-aceh,ACEH,ACEH SELATAN,2022,3743,15838559,ACEH,"MULTIPOLYGON (((97.97681 4.62750, 98.00173 4.6..."
2,16,Indonesia,1,indonesia-aceh,ACEH,ACEH TENGGARA,2022,3054,13436165,ACEH,"MULTIPOLYGON (((97.97681 4.62750, 98.00173 4.6..."
3,16,Indonesia,1,indonesia-aceh,ACEH,ACEH TIMUR,2022,7162,16710999,ACEH,"MULTIPOLYGON (((97.97681 4.62750, 98.00173 4.6..."
4,16,Indonesia,1,indonesia-aceh,ACEH,ACEH TENGAH,2022,5174,23631614,ACEH,"MULTIPOLYGON (((97.97681 4.62750, 98.00173 4.6..."


In [23]:
gdf_1.dtypes

cartodb_id                    int64
country                      object
id_1                          int64
slug                         object
state                        object
Kab/Kota                     object
Tahun                         int64
Regional GDP (IDR Bil)        int64
Regional GDP per Capita       int64
geo_name                     object
geometry                   geometry
dtype: object